# Widerstandsnetzwerke

In [4]:
# Bibliotheken importieren
import ipywidgets as widgets
from ipywidgets import Box, HBox, VBox, Layout, Image
from ipycanvas import MultiCanvas, Canvas, hold_canvas
from math import pi, sin, cos, atan, sqrt


# Farben definieren
colors = {
    "black" : "#000000",
    "U"     : "#3333cc",
    "I"     : "#e8082c",
    "R"     : "#00d000",
    "lgrey" : "#f0f0f0",
    "dgrey" : "#c0c0c0",
    "diel"  : "#99ff99",
    "white" : "#ffffff",
    "elec"  : "#ffff00"
}

# Konstanten definieren

y0_top  =  50
y0_dist = 125

# Vorzeichnen redundanter Grafiken
el_dict = {}

# Widerstand
img_R = Canvas(width=40, height=18)
img_R.stroke_style = colors["black"]
img_R.line_width = 3
img_R.begin_path()
img_R.move_to( 2,  2)
img_R.line_to(38,  2)
img_R.line_to(38, 16)
img_R.line_to( 2, 16)
img_R.line_to( 2,  2)
img_R.line_to( 3,  2)
img_R.stroke()
img_R.fill_style = colors["white"]
img_R.fill()
el_dict["R"] = (img_R, 20, 9)

# Knoten
img_node = Canvas(width=6, height=6)
img_node.fill_style = colors["black"]
img_node.fill_arc(3, 3, 3, 0, 2 * pi)
el_dict["node"] = (img_node, 3, 3)

# Schalter, offen, vertikal
img_vsw0 = Canvas(width=15, height=30)
img_vsw0.fill_style = "#ffffff"
img_vsw0.fill_rect(0, 0, 15, 30)
img_vsw0.draw_image(img_node, 9, 0)
img_vsw0.begin_path()
img_vsw0.move_to(13, 1)
img_vsw0.line_to(3, 27)
img_vsw0.stroke()
el_dict["Sv0"] = (img_vsw0, 12, 3)

# Schalter, offen, horizontal
img_hsw0 = Canvas(width=30, height=15)
img_hsw0.fill_style = "#ffffff"
img_hsw0.fill_rect(0, 0, 30, 15)
img_hsw0.draw_image(img_node, 0, 9)
img_hsw0.begin_path()
img_hsw0.move_to( 0, 13)
img_hsw0.line_to(27,  3)
img_hsw0.stroke()
img_hsw0.set_line_dash([3, 3])
img_hsw0.begin_path()
img_hsw0.move_to(4,  12)
img_hsw0.line_to(30, 12)
img_hsw0.stroke()
el_dict["Sh0"] = (img_hsw0, 15, 12)

# Schalter, geschlossen, vertikal
img_vsw1 = Canvas(width=6, height=30)
img_vsw1.draw_image(img_node, 0, 3)
img_vsw1.begin_path()
img_vsw1.move_to(3,  4)
img_vsw1.line_to(3, 30)
img_vsw1.stroke()
el_dict["Sv1"] = (img_vsw1, 3, 3)

# Schalter, geschlossen, horizontal
img_hsw1 = Canvas(width=30, height=15)
img_hsw1.fill_style = "#ffffff"
img_hsw1.fill_rect(0, 0, 30, 15)
img_hsw1.draw_image(img_node, 0, 9)
img_hsw1.begin_path()
img_hsw1.move_to(4,  12)
img_hsw1.line_to(30, 12)
img_hsw1.stroke()
img_hsw1.set_line_dash([3, 3])
img_hsw1.begin_path()
img_hsw1.move_to( 0, 13)
img_hsw1.line_to(27,  3)
img_hsw1.stroke()
el_dict["Sh1"] = (img_hsw1, 15, 12)

# Lampe
def img_GL(brightness=1, style=1, n=1):
    
    c = Canvas(width=40, height=40)
    
    # Grundfarbe weiß/gelb
    if brightness > 0:
        c.fill_style = "#ffff00"
    else:
        c.fill_style = "#ffffff"
        
    with hold_canvas(c):
        
        c.line_width = 1
        c.fill_arc(20, 20, 9, 0, 2 * pi)

        # Sonnenstrahlen
        if style == 1 and n>1 and brightness>0:
            
            for i in range(int(n*brightness)):
                
                if not(n == 1 and i == 0):

                    phi = pi / (4*int(n*brightness)) * (2*i + 1)
                    co  = cos(phi)
                    si  = sin(phi)
                    x0  = co * 10
                    x1  = co * (10 + 10 * brightness)
                    y0  = si * 10
                    y1  = si * (10 + 10 * brightness)
                    
                    c.stroke_style = "#ffb000"
                    c.line_width = 2
                    c.begin_path()
                    c.move_to(20 + x0, 20 + y0)
                    c.line_to(20 + x1, 20 + y1)
                    c.move_to(20 + x0, 20 - y0)
                    c.line_to(20 + x1, 20 - y1)
                    c.move_to(20 - x0, 20 + y0)
                    c.line_to(20 - x1, 20 + y1)
                    c.move_to(20 - x0, 20 - y0)
                    c.line_to(20 - x1, 20 - y1)
                    c.stroke()
                    
            c.fill_style = "#ffb000"
            #c.fill_arc(20, 20, 10 + 5*brightness, 0, 2*pi)
            
            c.fill_style = "#ffff00"
            c.fill_arc(20, 20, 10 + 4*brightness, 0, 2*pi)            

                    
                    
        # Ladebalken ohne/mit Unterteilung
        if style in [2, 3] and n>1 and brightness>0:
            c.line_join = "miter"
            c.fill_style = "#d0d0d0"
            c.fill_rect(5, 34, 30, 5)
            c.fill_style = "#ffff00"
            c.fill_rect(5, 34, 30 * brightness, 5)
            c.line_width = 1
            c.begin_path()
            c.move_to( 5, 34)
            c.line_to(35, 34)
            c.line_to(35, 39)
            c.line_to( 5, 39)
            c.line_to( 5, 34)
            c.stroke()
            
            c.line_width = 0.5
            
            if style == 2:
                for i in range(n):
                    c.begin_path()
                    c.move_to(5 + i*(30/n), 34)
                    c.line_to(5 + i*(30/n), 39)
                    c.stroke()
                    
            if style == 3:
                c.begin_path()
                c.move_to(5 + brightness * 30, 34)
                c.line_to(5 + brightness * 30, 39)
                c.stroke()
        
        c.line_width = 1.25
        #c.fill_style = "#ffffff"
        #c.fill_arc(20, 20, 9, 0, 2 * pi)
        c.stroke_style = colors["black"]
        c.stroke_arc(20, 20, 9, 0, 2 * pi)
        c.stroke_style = colors["black"]
        c.begin_path()
        c.move_to(14, 14)
        c.line_to(26, 26)
        c.stroke()
        c.begin_path()
        c.move_to(26, 14)
        c.line_to(14, 26)
        c.stroke()
    return (c, 20, 20)

el_dict["GL"] = img_GL

#tC = Canvas(width=600, height=300)
#display(tC)

def update_L(*args):
    
    with hold_canvas(tC):
        
        tC.clear()
        
        n = tn.value
        
        for s in range(4):
            
            for b in range(n + 1):
                
                tC.draw_image(img_GL(brightness = (b)/n,
                                     style      = s,
                                     n          = n)[0],
                              10+50*b, 50*(s-1))
        
        #tC.draw_image(img_GL(brightness = tW.value, style = 4, n=n)[0])

#tW = widgets.FloatSlider(min=0, max=1, step=0.01)
#tW.observe(update_L, "value")
#tn = widgets.IntSlider(min=1, max=10, step=1, value=1)
#tn.value = 1
#tn.observe(update_L, "value")
#display(tn)
#display(tW)
#update_L()

In [2]:
class GridElement():
    
    def __init__(self, parent, value="R", pointer=[None], backpointer=None):
        
        #self.etype  = etype
        self.parent = parent
        
        self.menu = widgets.Dropdown(options = [["-",                     "-"],
                                                ["Widerstand",            "R"],
                                                ["Lampe",                 "GL"],
                                                ["Schalter, offen",       "Sh0"],
                                                ["Schalter, geschlossen", "Sh1"],
                                                ["Reihenschaltung",       "ser"],
                                                ["Parallelschaltung",     "par"]],
                                     layout  = Layout(width="145px"),
                                     value   = value)
        
        self.menu.observe(self.update, "value")
        
        self.pointer     = pointer
        self.backpointer = backpointer

    
    
    def update(self, *args):
                
        def f_pass(): pass
        
        def f_serial():
            self.menu.value = "R"
            self.pointer = [GridElement(parent      = self.parent,
                                        value       = self.menu.value,
                                        backpointer = self)]
            
        
        def f_parallel():
            self.menu.value = "R"
            
            self.backpointer.pointer.append(GridElement(parent      = self.parent,
                                                        value       = self.menu.value,
                                                        backpointer = self.backpointer))
        
        def f_remove():
            self.backpointer.pointer.remove(self)
            #self.pointer[0].backpointer = self.backpointer
        
        switcher = {
            "ser"    : f_serial,
            "par"    : f_parallel,
            "-"      : f_remove
        }
        
        switch_func = switcher.get(self.menu.value, f_pass)
        switch_func()
        
        self.parent.reconstruct()
        
    
    
    def draw_arrow(self, layer, xy, angle, length, color="#000000", line_width=2, tip_width=10, hold=True):
        
        sin_a = sin((angle + 90) * pi / 180)
        cos_a = cos((angle + 90) * pi / 180)

        if length < 15:
            tip_length = 15 + (length - 15)
            width = tip_width / 2 * length / 15
        else:
            tip_length = 15
            width = tip_width / 2
        line_end = (xy[0] + (length - tip_length) * sin_a,
                    xy[1] + (length - tip_length) * cos_a)        
        
        def do_draw(layer):
            
            # Pfeilschaft
            layer.line_width = line_width
            layer.stroke_style = color
            layer.fill_style   = color
            layer.begin_path()
            layer.move_to(xy[0], xy[1])
            layer.line_to(line_end[0], line_end[1])
            layer.stroke()
            layer.close_path()
            
            # Pfeilspitze
            layer.move_to(xy[0] + length * sin_a,
                          xy[1] + length * cos_a)
            layer.line_to(line_end[0] + width * sin((angle + 180) * pi / 180),
                          line_end[1] + width * cos((angle + 180) * pi / 180))
            layer.line_to(line_end[0] - width * sin((angle + 180) * pi / 180),
                          line_end[1] - width * cos((angle + 180) * pi / 180))
            layer.fill()
        
        if hold:
            with hold_canvas(layer):
                do_draw(layer)
        else:
            do_draw(layer)
    
    
    
    def draw(self, comp, grid, x, y):
        
        # x, y in Gitterkoordinaten
        
        if self.menu:
            
            if self.menu.value == "GL":
                el = el_dict[self.menu.value](brightness = self.rank / self.parent.n_ranks,
                                              n = self.parent.n_ranks)
            
            else:
                el = el_dict[self.menu.value]
        
        else:
            el = el_dict["Sv0"]
        
        x_dist   = 162.5
        x0_dist  = 106.25
        x_offset = el[1]
        
        y_dist   = 50
        y_offset = el[2]
        
        grid.begin_path()
        
        ###grid.stroke_style = "#000000"

        if x != -1:
            grid.move_to(    x * x_dist + 25, y * y_dist + y0_dist)
            # Horizontale
            grid.line_to((x+1) * x_dist + 25, y * y_dist + y0_dist)
        else:
            grid.move_to((x+1) * x_dist + 25, y * y_dist + y0_dist)
            
        
        
        """if x == -1:
            #x = 0
            grid.stroke_style = "#ff0000"
        else:
            grid.stroke_style = "#000000"
        """
        
        # Kein nachgeschaltetes Element
        if self.pointer == [None]:
            grid.line_to(675, y     * y_dist + y0_dist)
            if self.y > 0:
                grid.line_to(675, (y-1) * y_dist + y0_dist)
                grid.draw_image(img_node, 675 - 3, (y-1) * y_dist + y0_dist - 3)
        
        # Mehrere nachgeschaltete Elemente
        elif len(self.pointer) > 1:
            y_max = 0
            for pi in range(len(self.pointer)):
                p = self.pointer[pi]
                if p != self.pointer[-1]:
                    grid.draw_image(img_node, (x+1) * x_dist + 25 - 3,     p.y * y_dist + y0_dist - 3)
                if p.y > y_max: y_max = p.y
            # Vertikale
            grid.line_to(             (x+1) * x_dist + 25,     y_max * y_dist + y0_dist)
        
        grid.stroke()
        
        comp.draw_image(el[0], x*x_dist + x0_dist - x_offset, y*y_dist + y0_dist - y_offset)
        
        # --- Labels zeichnen
        comp.font = '14px serif'
        comp.text_align = "left"
        comp.fill_style = colors["I"]
        pre = "I"
        if self.menu:
            pre += self.menu.value
            if pre[1] != "S":
                comp.fill_style = colors["I"]
                self.draw_arrow(comp, (x*x_dist + x0_dist - 65,  y*y_dist + y0_dist), 0, 40, colors["I"], line_width=3, tip_width=10, hold=False)
            else:
                pre = pre[1]
                comp.fill_style = colors["black"]
            comp.fill_text(pre + str(self.i), x*x_dist + x0_dist - 65, y*y_dist + y0_dist - 15)
                            
    
    
class Grid(GridElement):
    
    def __init__(self, U=1):
        
        layers = ["bg",
                  "grid",
                  "components"]
        lx = 700
        ly = 450
        
        self.canvas = MultiCanvas(len(layers), width=lx, height=ly)
        self.c = {}
        self.U = U
        for li in range(len(layers)):
            self.c[layers[li]] = self.canvas[li]
        
        self.menu    = None #GridElement(value="hsw0", parent=self, pointer=[None], backpointer=self)
        self.pointer = [GridElement(value="R", parent=self, pointer=[None], backpointer=self)]
                
        self.out = widgets.Output()
        
        self.draw_background()
        self.reconstruct()
        
        display(self.canvas)
        display(self.out)
    
    
    
    def draw_background(self):
        
        bg = self.c["bg"]
        
        bg.begin_path()
        bg.move_to( 25, y0_dist)
        bg.line_to( 25, y0_top)
        bg.line_to(675, y0_top)
        bg.line_to(675, y0_dist)
        bg.stroke()
        bg.line_width = 1.25
        bg.stroke_arc(350, 50, 20, 0, 2 * pi)
        
        bg.font = '14px serif'
        bg.text_align = "left"
        bg.fill_style = colors["U"]
        self.draw_arrow(bg, (322.5, 15), 0, 60, colors["U"], line_width=3, tip_width=10, hold=False)
        bg.fill_text("U0", 290, 20)
    
    
    
    def reconstruct(self):
        
        # --- Indizierung --- #
        
        print()
        
        matrix = {}
        
        grid = self.c["grid"]
        comp = self.c["components"]
        
        self.el_counter = {}
        
        #####################################################################################################################
        ############################################### --- REKURSION --- ###################################################
        #####################################################################################################################
        
        def rec(el, x=-1, y=0):
                        
            if 1:#el != self:
                
                # Element in Matrix
                matrix[y, x] = el
                
                # Koordinaten in Element
                el.x, el.y = x, y
                
                # Zähler für Bauteilindizes
                if el.menu:
                    
                    # Gemeinsamer Index für Schalter offen/geschlossen
                    if el.menu.value[0] == "S":
                        etype = "S"
                    else:
                        etype = el.menu.value
                        
                    if etype in self.el_counter:
                        self.el_counter[etype] += 1
                    else:
                        self.el_counter[etype]  = 1
                    el.i = self.el_counter[etype]
                else:
                    el.i = ""
            
            # R aus Bauteiltyp bestimmen
            R = 0
            if el.menu:
                
                # Widerstand (R für alle Widerstände 1)
                if el.menu.value == "R":
                    
                    R = 1
                
                # Sonderfall offener Schalter (R = -2)
                if el.menu.value == "Sh0":
                    
                    R = -2
                
            el.R = R
            
            # Nicht mit Masse verbunden
            if el.pointer != [None]:
                
                ys = []
                R_KW = 0
                
                for i in range(len(el.pointer)):
                    
                    e = el.pointer[i]
                    
                    # Positionsbestimmung (Raster)
                    if len(el.pointer) <= 1 or i == 0:
                        yd = y
                    else:
                        yd = y + 1
                    
                    # Rekursion
                    answer = rec(e, x=x+1, y=yd)
                    e.Rg = answer[1]
                    
                    #if el.menu:
                    #    print("R:" , e.R)
                    #    print("Rg:", e.Rg)
                    #    print("Ir:", e.R/e.Rg)
                    
                    if e.Rg != "-2" and el.R != "-2":
                    
                        try:

                            if R_KW != -1:

                                R_KW += 1 / e.Rg

                        except ZeroDivisionError:

                            # print("boom!")
                            shorts.append(e)

                            R_KW = -1
                            # Strom durch Nachbarn mit R>0 muss dann 0 sein!

                    y = answer[0]
                    ys.append(y)
                    
                if R_KW == -1 and el.menu:
                    R_KW =  0
                
                # Reset y
                y -= yd
                
                print("R_KW", R_KW)
                if not(e.Rg == -2 and len(el.pointer) == 1):
                    
                    try:
                        print("x:", x, "\ty:", y, "\tR:", R + 1/R_KW)
                        return max(ys), R + 1/R_KW

                    except ZeroDivisionError:
                        print("x:", x, "\ty:", y, "\tR:", R)
                        #print("para boom!")
                        return max(ys), R
                
                else:
                    return max(ys), -2
                            
            # Mit Masse verbunden
            else:
                
                print("x:", x, "\ty:", y, "\tR:", R)
                return y, R
            
        #####################################################################################################################
        ############################################### --- REKURSION --- ###################################################
        #####################################################################################################################
        
        shorts = []
        
        self.R  = rec(self)[1]
        self.Rg = self.R
        print("R_ges = ", self.R, "\n")
        print()
        
        def rec2(el, U_in=self.U, I=0):
            
            for i in range(len(el.pointer)):
                
                e = el.pointer[i]
                
                # Nicht mit Masse verbunden
                if e != None:
                    
                    print("I_alt,", I)
                    print(e.Rg)
                    
                    # Bauteiltyp bestimmen
                    etype = e.menu.value
                    
                    # offener Schalter in Zweig
                    if e.Rg == -2:
                        Ir    = 0
                        U_out = 0
                    
                    # Widerstand
                    elif etype == "R":
                        Ir    = U_in / e.Rg
                        U_out = U_in * (1 - e.R / e.Rg)
                        
                    elif e.Rg > 0:
                        Ir, U_out = U_in / e.Rg, U_in
                    
                    else:
                        if not(el.menu): print("Kurzer!")
                        Ir, U_out = I / len(el.pointer), U_in
                    
                    e.I = Ir
                        
                    if 1:#etype == "R" or etype [:2] == "GL":

                        print("-----")
                        print(" " + etype + str(e.i))
                        print("-----")
                        print("U_in:\t", U_in)
                        print("R:\t", e.R)
                        print("Rg:\t", e.Rg)
                        print("I:\t", Ir)
                        print("U_out:\t", U_out)
                        #if e in shorts: print("SHORTED!")
                        print()
                        
                        # Lampen für Rangliste erfassen
                        if etype [:2] == "GL":
                            if Ir in list(lamps.keys()):
                                lamps[Ir].append(e)
                            else:
                                lamps[Ir] = [e]

                    rec2(e, U_out, Ir)
        
        lamps = {}
        rec2(self)
        #print(lamps)
        
        #print("Shorts")
        #print(shorts)
        
        #for i in self.pointer
        
        index = list(matrix)
        
        # --- Darstellung --- #
        
        x_last = -1
        y_last =  0
        
        rows = []
        row  = []
        
        grid.clear()
        comp.clear()
        
        # Helligkeits-Rangliste erstellen
        keys = list(lamps.keys())
        keys.sort()
        #print(keys)
        keys_done = []
        rank = 1
        self.n_ranks = len(keys)
        for key in keys:
            #print(rank, lamps[key])
            for l in lamps[key]:
                if l.I == 0:
                    l.rank = 0
                else:
                    l.rank = rank
            if not(key in keys_done):
                rank += 1
        
        # Maximalwerte finden (später für schönere Darstellung)        
        #for i in index:
        #    if i[1] > x_max:
        #        x_max = i[1]
        #        
        #y_max = index[-1][0]
        
        # Elemente darstellen
        
        for i in index:
            
            x = i[1]
            y = i[0]
            
            if y != y_last:
                rows.append(HBox(row))
                row    = []
                x_last = -1

            # Zeilenabstand
            if (x - x_last) > 1:
                for j in range(x - x_last - 1):
                    row.append(Box(layout=Layout(width="149px")))

            if x > -1:
                row.append(matrix[i].menu)
             
            with hold_canvas(grid):
                matrix[i].draw(comp, grid, x, y)
            
            x_last = x
            y_last = y
                
        rows.append(HBox(row))
        
        with self.out:
            self.out.clear_output()
            display(VBox(rows))



G = Grid(U=0)


x: 0 	y: 0 	R: 1
R_KW 1.0
x: -1 	y: 0 	R: 1.0
R_ges =  1.0 


I_alt, 0
1
-----
 R1
-----
U_in:	 0
R:	 1
Rg:	 1
I:	 0.0
U_out:	 0.0



MultiCanvas(height=450)

Output()